# Battle of Neighborhoods Code

#### Let's first download all the dependencies that we need in order to accomplish this project

In [1]:
! pip install folium==0.5.0
!pip install geocoder
!pip install geopy

     |████████████████████████████████| 79 kB 8.3 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=efefc21782a130800f96be712afa6f46ba961c2a3b902bdd99235314c1b839f4
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
     |████████████████████████████████| 98 kB 9.3 MB/s  eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
import json
import matplotlib as mpl
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup
import geocoder
from geopy.geocoders import Nominatim

#### Now Let's assemble the data first and make a cleaned dataframe for the project

In [3]:
webtext = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bs = BeautifulSoup(webtext,'xml')
table = bs.find('div', class_='mw-parser-output')
table_rows = table.find_all('tr')
#Let's clean it now
cleaned_data=[]
for rows in table_rows:
    temp=[]
    for i in rows.find_all('td'):
        temp.append(i.text.strip())
    cleaned_data.append(temp)
df1=pd.DataFrame(data=cleaned_data)
df2=df1[[0,1,2]]
df2.rename( columns={0:'PostalCode', 1:'Borough', 2:'Neighborhood'}, inplace=True )
df2=df2[~df2['Borough'].isnull()]
df2=df2[df2.Borough!='Not assigned']
df2=df2[:-4]
df2=df2.reset_index(drop=True, inplace= False)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:

import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9bc7891248c24f4681195e5fd8304ff3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='hjcfypfLhxv4qrM2JJW2O3pbaVUHv4o8leAEIaQJEscO',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_9bc7891248c24f4681195e5fd8304ff3.get_object(Bucket='applieddatasciencecapstoneproject-donotdelete-pr-5smw9igoycyfvy',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
mergedDF=pd.merge(left=df2, right=df_data_1, left_on='PostalCode', right_on='Postal Code')
del mergedDF['Postal Code']
mergedDF.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [7]:
#making a backup dataframe of cleaned data
toronto_data_backup=mergedDF

In [8]:
downtown_df=mergedDF.loc[mergedDF['Borough']=='Downtown Toronto']

In [11]:
downtown_df.shape

(19, 5)

In [10]:
address = 'Downtown Toronto, Canada'
geolocator = Nominatim(user_agent="Downtown Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.6563221, -79.3809161.


#### Let's have a look at the map of downtown toronto

In [12]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, nei in zip(downtown_df['Latitude'], downtown_df['Longitude'], downtown_df['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown